In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ForwardKinematics import runFK
import numpy as np

import time
from Arm_Lib import Arm_Device

def get_inputs():
    global bypass
    bypass = False
    while True:
        user_input = input("Please enter 5 integers separated by spaces: ")
        if user_input == "up":
            return [90, 90, 90, 90, 90]
        elif user_input == "staircase":
            return [90, 0, 180, 0, 90]
        elif user_input == "out":
            return [90, 0, 90, 90, 90]
        elif user_input == "buzzer":
            arm.Arm_Buzzer_On(5)
            bypass = True
            return qlast
        elif user_input == "read":
            print("measured angles: [{}, {}, {}, {}, {}]".format(
            arm.Arm_serial_servo_read(1),
            arm.Arm_serial_servo_read(2),
            arm.Arm_serial_servo_read(3), 
            arm.Arm_serial_servo_read(4), 
            arm.Arm_serial_servo_read(5) ))
            bypass = True
            return qlast
         
        
        try:
            # Split the input string into a list and convert each element to an integer
            integer_array = list(map(int, user_input.split()))
            
            # Check if the length of the array is 5
            if len(integer_array) == 5:
                return integer_array
            else:
                print("Error: You must enter exactly 5 integers.")
        except ValueError:
            print("Error: Please enter only integers.")
     
    
arm = Arm_Device()
qlast = None
bypass = False
time.sleep(1)
            
while True:
    arm.Arm_RGB_set(0, 255, 0)
    q_input = get_inputs()
    q = np.array([q_input]).T
    if bypass:
        continue
    
    if runFK(q):
        print("\nmoving to ", [int("".join([str(angle) for angle in arr])) for arr in q])
        arm.Arm_RGB_set(255, 0, 0)
        arm.Arm_serial_servo_write6(*q, 0, 1000)
        time.sleep(1.5)
        qlast = q